In [8]:
from dotenv import load_dotenv
import os
from hera.workflows import EmptyDirVolume, models, script, Artifact, Parameter, DAG, Steps, Step, NoneArchiveStrategy, Workflow
from hera.shared import global_config

load_dotenv("/home/otto/s1_zarr/.env")

True

In [9]:
global_config.host = "https://dev.services.eodc.eu/workflows/"
global_config.namespace = "inca"
global_config.token = os.getenv("argo_token")
global_config.image = "ghcr.io/oscipal/image_zarr:latest"

In [10]:
@script(outputs=Artifact(name="output-artifact", path="/tmp/INCA.nc", archive=NoneArchiveStrategy()))
def create_artifact():
    from urllib.request import urlretrieve
    import datetime
    import os 

    artifact_path = "/tmp"

    url = f"https://public.hub.geosphere.at/datahub/resources/inca-v1-1h-1km/filelisting/RR/INCAL_HOURLY_RR_202506.nc"
    urlretrieve(url, os.path.join(artifact_path, "INCA.nc"))

In [ ]:
@script(inputs=Artifact(name="input-artifact", path="/tmp/INCA.nc"))
def consume_artifact():
    import xarray as xr

    artifact_path="/tmp/INCA.nc"

    ds = xr.open_dataset(os.path.join(artifact_path, "INCA.nc")).isel(time=1, x=1, y=1)["RR"].load()
    print(ds.values)

In [43]:
with Workflow(
    generate_name="artifact-storage-",
    entrypoint="workflow"
) as w:
    with DAG(name="workflow"):
        download = create_artifact()
        consume = consume_artifact(arguments=[download.get_artifact("output-artifact").with_name("input-artifact")])

        download >> consume

In [44]:
w.create()

Workflow(api_version=None, kind=None, metadata=ObjectMeta(annotations=None, cluster_name=None, creation_timestamp=Time(__root__=datetime.datetime(2025, 7, 24, 9, 16, 44, tzinfo=datetime.timezone.utc)), deletion_grace_period_seconds=None, deletion_timestamp=None, finalizers=None, generate_name='artifact-storage-', generation=1, labels={'workflows.argoproj.io/creator': 'system-serviceaccount-default-jenkins'}, managed_fields=[ManagedFieldsEntry(api_version='argoproj.io/v1alpha1', fields_type='FieldsV1', fields_v1=FieldsV1(), manager='argo', operation='Update', subresource=None, time=Time(__root__=datetime.datetime(2025, 7, 24, 9, 16, 44, tzinfo=datetime.timezone.utc)))], name='artifact-storage-hm5w6', namespace='inca', owner_references=None, resource_version='315382741', self_link=None, uid='a5aa2bda-11e2-4d23-87cc-0abadd75d8b0'), spec=WorkflowSpec(active_deadline_seconds=None, affinity=None, archive_logs=None, arguments=Arguments(artifacts=None, parameters=None), artifact_gc=None, artif